In [51]:
import numpy as np
import torch
import glob
import re

from NEXT_graphNN.utils.data_loader import weights_loss, load_graph_data, split_dataset, LabelType, NetArchitecture
from NEXT_graphNN.utils.train_utils import train_net, predict_gen, train_one_epoch

from torch_geometric.nn.models import GraphUNet

In [31]:
labeltype = LabelType.Segmentation
netarch   = NetArchitecture.GraphUNet

## Net architecture variables
init_features = 1 
hidden_dim    = 20
nclass        = 2
depth         = 4
pool_ratio    = 0.3

# Net weights to load
saved_weights = None
# saved_weights = 'path/to/trained/weights' ##would load certain weights for the net

## Data
train_perc = 0.9

## Train
nepoch          = 1
train_batch     = 50
valid_batch     = 50
checkpoint_dir  = '/mnt/lustre/scratch/nlsas/home/usc/ie/mpm/NEXT100/graph_toy_experiment/train/checkpoints/'
tensorboard_dir = '/mnt/lustre/scratch/nlsas/home/usc/ie/mpm/NEXT100/graph_toy_experiment/train/tensorboard/'
num_workers     = 3
#Loss
LossType        = 'CrossEntropyLoss'
#Optimizer
lr           = 1e-2
betas        = (0.9, 0.999)
eps          = 1e-6
weight_decay = 0

In [27]:
path = sorted(glob.glob('/mnt/lustre/scratch/nlsas/home/usc/ie/mpm/NEXT100/graph_toy_experiment/prod/*.pt'), 
              key = lambda x: int(re.findall(r'\d+', x)[-1]))

In [3]:
data = [graph for f in path for graph in torch.load(f) if graph.edge_index.numel() != 0]

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [45]:
net = GraphUNet(init_features, 
                hidden_dim,
                nclass, 
                depth,
                pool_ratio).to(device)

model_uses_batch = True

In [9]:
train_data, valid_data, test_data = split_dataset(data, train_perc=0.9)

In [41]:
weights = weights_loss(train_data, labeltype, nclass = 2)

In [43]:
criterion = torch.nn.CrossEntropyLoss(weight = torch.Tensor(weights).to(device))

In [46]:

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()),
                                     lr = lr,
                                     betas = betas,
                                     eps = eps,
                                     weight_decay = weight_decay)

In [49]:
train_net(nepoch = nepoch,
                  train_data = train_data,
                  valid_data = valid_data,
                  train_batch_size = train_batch,
                  valid_batch_size = valid_batch,
                  net = net,
                  device = device,
                  optimizer = optimizer,
                  criterion = criterion,
                  label_type = labeltype,
                  nclass = nclass,
                  model_uses_batch = model_uses_batch,
                  checkpoint_dir  = checkpoint_dir,
                  tensorboard_dir = tensorboard_dir,
                  num_workers = num_workers)

/mnt/netapp2/Store_uni/home/usc/ie/mpm/conda/envs/3.8-pytorch-env/lib/python3.8/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)

In [58]:
from torch_geometric.loader import DataLoader


loader_train = DataLoader(train_data,
                            batch_size = train_batch,
                            shuffle = True,
                            num_workers = num_workers,
                            drop_last = True,
                            pin_memory = False)

/mnt/netapp2/Store_uni/home/usc/ie/mpm/conda/envs/3.8-pytorch-env/lib/python3.8/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [59]:
loader_train

In [63]:
for batch in loader_train:

    # Pass the batch to device (cuda)
    batch = batch.to(device)

    # Zero grad the optimizer
    optimizer.zero_grad()
    break

In [83]:
batch.x.shape

torch.Size([1006])

In [93]:
batch.x.type(torch.float).unsqueeze(1).shape

torch.Size([1006, 1])

In [94]:
batch.edge_index.shape

torch.Size([2, 1396])

In [75]:
from torch_geometric.nn import GCNConv

In [92]:
GCNConv(1, 20)(batch.x.type(torch.float).unsqueeze(1), batch.edge_index)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat2 in method wrapper_mm)

In [67]:
out = net.forward(batch.x.type(torch.float), batch.edge_index, batch.batch) 

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)

In [60]:
train_one_epoch(0, net, loader_train, device, optimizer, criterion, labeltype, nclass = nclass, model_uses_batch = model_uses_batch)

/mnt/netapp2/Store_uni/home/usc/ie/mpm/conda/envs/3.8-pytorch-env/lib/python3.8/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)